In [1]:
# imports
import os
import pandas as pd
import numpy as np
import requests
import json



# Foursquare

Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

In [7]:
#Loading bike data
bikes_df = pd.read_csv('bikes_df.csv')
# Accessing the Foursquare API key from environment variables
key = os.environ.get('foursquare_api')
#confirm that data has loaded
bikes_df

,Station Name,Latitude,Longitude,Bikes Available
0,10th & Cambie,49.262487,-123.114397,20
1,Yaletown-Roundhouse Station,49.274566,-123.121817,11
2,Dunsmuir & Beatty,49.279764,-123.110154,13
3,12th & Yukon (City Hall),49.260599,-123.113504,8
4,8th & Ash,49.264215,-123.117772,11
...,...,...,...,...
243,War Memorial Gym,49.266579,-123.248549,13
244,Pharmaceutical Sciences,49.262620,-123.243100,3
245,Georgia & Main,49.278529,-123.100017,0
246,Bute & Davie,49.282047,-123.133057,17


In [37]:
# Define headers for Foursquare API requests
headers = {
    "accept": "application/json",
    "Authorization": key
}
# API endpoint
url = "https://api.foursquare.com/v3/places/search"

In [38]:
 params = {
        'query': 'Cineplex Theatres',
        'll': f"{bikes_df.loc[0,'Latitude']},{bikes_df.loc[0, 'Longitude']}",
        'radius': 1000,
        'limit': 4,      # Limit the number of results
    
        'fields': 'name,location,geocodes,categories,distance,rating,tel,hours,stats,features'
    }
   
params    

{'query': 'Cineplex Theatres',
 'll': '49.262487,-123.114397',
 'radius': 1000,
 'limit': 4,
 'fields': 'name,location,geocodes,categories,distance,rating,tel,hours,stats,features'}

In [39]:
response = requests.get(url, params=params, headers=headers)

In [40]:
# the code is extracting the latitude and longitude values for the first row of the bikes_df DataFrame
bikes_df.loc[0,'Latitude']
bikes_df.loc[0, 'Longitude']

-123.114397

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [41]:
# Point of interest was Cineplex Theatres
response_list = []

for station in range(len(bikes_df)):
    latitude = bikes_df.loc[station,'Latitude']
    longitude =bikes_df.loc[station, 'Longitude']

    url = "https://api.foursquare.com/v3/places/search"

    headers = {"Accept": "application/json",
                 "Authorization": key}

    params = {
        'query': 'Cineplex Theatres',
        'll': f"{latitude},{longitude}",
        'radius': 1000,
        'limit': 4,      
    
        'fields': 'name,location,geocodes,categories,distance,rating,tel,hours,stats,features'
    }


    response = requests.get(url, params=params, headers=headers)

    if response.status_code == 200:
        data_json = response.json()
        response_list.append(data_json)
    else:
        print(f"Request was unsuccessful: {response.status_code}")
        
unnested_data = [
    {
    "Latitude": response["geocodes"]["main"].get("latitude"),
    "Longitude": response["geocodes"]["main"].get("longitude"),
    "name": response.get("name"),
    "distance": response.get("distance"),
    "address": response["location"].get("formatted_address"),
    "telephone": response.get("tel"),
    "rating": response.get("rating"),
    "open_true": response["hours"].get("open_now")
    }
  for result in response_list
  for response in result.get("results", [])
]

Put your parsed results into a DataFrame

In [46]:
Cineplex_Theatres = pd.DataFrame(unnested_data)

Cineplex_Theatres

,Latitude,Longitude,name,distance,address,telephone,rating,open_true
0,49.263025,-123.113913,Pacific Ballet Theatre School,69,"456 Broadway W, Vancouver BC V5Y 1R3",(604) 873-5024,NaN,False
1,49.254508,-123.114891,Cineplex Inc,887,"3440 Cambie St, Vancouver BC V5Z 2W8",(604) 709-3456,NaN,False
2,49.281878,-123.124201,Cineplex Inc,830,"900 Burrard St, Vancouver BC V6Z 3G5",(604) 630-1416,NaN,True
3,49.280570,-123.119384,Vancouver Civic Theatres,690,"823 Seymour St, Vancouver BC V6B 3L4",(604) 665-3050,NaN,False
4,49.281089,-123.112776,Vancouver Civic Theatres,977,"600 Hamilton St, Vancouver BC V6B 3L4",(604) 665-3050,NaN,False
...,...,...,...,...,...,...,...,...
328,49.281878,-123.124201,Cineplex Inc,642,"900 Burrard St, Vancouver BC V6Z 3G5",(604) 630-1416,NaN,True
329,49.280300,-123.106705,Cineplex Cinemas,609,"88 W Pender St (btwn Abbott & Carrall), Vancou...",(604) 806-0799,6.7,False
330,49.282767,-123.109268,Kokoro Dance Theatre Society,661,"207 Hastings St W, Vancouver BC V6B 1H7",(604) 662-7441,NaN,True
331,49.281089,-123.112776,Vancouver Civic Theatres,383,"600 Hamilton St, Vancouver BC V6B 3L4",(604) 665-3050,NaN,False


In [113]:
Cineplex_Theatres.to_csv('Cineplex_Theatres.csv', index=False)

# Yelp

Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

In [63]:
# Accessing the Yelp API key from environment variables
key = os.environ.get('yelp_api')

In [68]:
url_yelp = "https://api.yelp.com/v3/businesses/search"
#api endpoint

In [84]:
headers = {"accept": "application/json",
           "Authorization": f"Bearer {key}"}

In [85]:
 params = {
           'latitude': latitude,
           'longitude': longitude,
           'term': 'Cineplex Theatres',
           'radius': 1000,
             } 

In [86]:
latitude, longitude

(49.279764, -123.110154)

In [87]:
response = requests.get(url_yelp, params=params, headers=headers)

In [88]:
response.json()

{'businesses': [{'id': 'tlPUV1xUtBsnPGQT66duXg',
   'alias': 'cineplex-odeon-international-village-vancouver',
   'name': 'Cineplex Odeon International Village',
   'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/qSWGuZS-lYdGmSVwnL6_Pg/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/cineplex-odeon-international-village-vancouver?adjust_creative=hFoJK0_8pC7raHFV2qnuMw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=hFoJK0_8pC7raHFV2qnuMw',
   'review_count': 107,
   'categories': [{'alias': 'movietheaters', 'title': 'Cinema'}],
   'rating': 4.0,
   'coordinates': {'latitude': 49.28029, 'longitude': -123.10675},
   'transactions': [],
   'location': {'address1': '88 W Pender Street',
    'address2': '',
    'address3': None,
    'city': 'Vancouver',
    'zip_code': 'V6B 6N9',
    'country': 'CA',
    'state': 'BC',
    'display_address': ['88 W Pender Street',
     'Vancouver, BC V6B 6N9',
     'Canada']},
   'phone': '+16048060799',
   'dis

In [89]:
bikes_df.loc[0,'Latitude']
bikes_df.loc[0, 'Longitude']

-123.114397

Parse through the response to get the POI (such as restaurants, bars, etc) details you want (ratings, name, location, etc)

In [95]:
#POI was Cineplex Theatres
y_response_list = []
maximum_request = 300
request_count = 0

for index, row in bikes_df.iterrows():
    if request_count >= maximum_request:
        break

    latitude = row['Latitude']
    longitude = row['Longitude']

    url_yelp = "https://api.yelp.com/v3/businesses/search"
    headers = {"Accept": "application/json",
                 "Authorization": f"Bearer {key}"}

    params = {
        'latitude': latitude,
        'longitude': longitude,
        'term': 'Cineplex Theatres',
        'radius': 1000,
    }

    response_yelp = requests.get(url=url_yelp, params=params, headers=headers)

    if response_yelp.status_code == 200:
        yelp_data_json = response_yelp.json()
        y_response_list.append(yelp_data_json)
        request_count += 1
    else:
        print(f"Request failed: {response_yelp.status_code}")


    

    




In [98]:
yelp_unnested_data = [
    {
    "name": response.get("name"),
    "distance": response.get("distance"),
    "address": response["location"].get("display_address"),
    "telephone": response.get("display_phone"),
    "rating": response.get("rating"),
    "review_counts": response.get("review_count"),
    "closed": response.get("is_closed")
    }
  for result in y_response_list
  for response in result.get("businesses", [])
]

Put your parsed results into a DataFrame

In [100]:
# Your code to create Yelp_Cineplex_Theatres DataFrame
Yelp_Cineplex_Theatres = pd.DataFrame(unnested_data)

In [101]:
Yelp_Cineplex_Theatres

,Latitude,Longitude,name,distance,address,telephone,rating,open_true
0,49.263025,-123.113913,Pacific Ballet Theatre School,69,"456 Broadway W, Vancouver BC V5Y 1R3",(604) 873-5024,NaN,False
1,49.254508,-123.114891,Cineplex Inc,887,"3440 Cambie St, Vancouver BC V5Z 2W8",(604) 709-3456,NaN,False
2,49.281878,-123.124201,Cineplex Inc,830,"900 Burrard St, Vancouver BC V6Z 3G5",(604) 630-1416,NaN,True
3,49.280570,-123.119384,Vancouver Civic Theatres,690,"823 Seymour St, Vancouver BC V6B 3L4",(604) 665-3050,NaN,False
4,49.281089,-123.112776,Vancouver Civic Theatres,977,"600 Hamilton St, Vancouver BC V6B 3L4",(604) 665-3050,NaN,False
...,...,...,...,...,...,...,...,...
328,49.281878,-123.124201,Cineplex Inc,642,"900 Burrard St, Vancouver BC V6Z 3G5",(604) 630-1416,NaN,True
329,49.280300,-123.106705,Cineplex Cinemas,609,"88 W Pender St (btwn Abbott & Carrall), Vancou...",(604) 806-0799,6.7,False
330,49.282767,-123.109268,Kokoro Dance Theatre Society,661,"207 Hastings St W, Vancouver BC V6B 1H7",(604) 662-7441,NaN,True
331,49.281089,-123.112776,Vancouver Civic Theatres,383,"600 Hamilton St, Vancouver BC V6B 3L4",(604) 665-3050,NaN,False


In [102]:
Yelp_Cineplex_Theatres.to_csv('Yelp_Cineplex_Theatres.csv', index=False)

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

The two APIs were comparable in terms of accessing Cineplex theatre data. This is shown below 

In [106]:
# Check for duplicating values to compare with Yelp as they both have 333 rows * 8 cols
Cineplex_Theatres.duplicated().sum()

7

In [107]:
# Same check on Yelp as with Foursquare above
Yelp_Cineplex_Theatres.duplicated().sum()


7

In [ ]:
The duplicates in this case have the same latitude, longitude, address, the difference is the distance

Get the top 10 restaurants according to their rating

In [111]:

#I had initially queried for entertainment in the form of cinema; the top 10 cinemas from Foursquare API are as follows;
top10cinema = Cineplex_Theatres.sort_values(by='rating', ascending=False).head(10)[['name', 'distance', 'address', 'telephone', 'rating', 'open_true']]
top10cinema

,name,distance,address,telephone,rating,open_true
168,Bard on the Beach Theatre Society,277,"1100 Chestnut St, Vancouver BC V6J 3J9",(604) 737-1551,8.2,True
94,Bard on the Beach Theatre Society,591,"1100 Chestnut St, Vancouver BC V6J 3J9",(604) 737-1551,8.2,True
244,Bard on the Beach Theatre Society,550,"1100 Chestnut St, Vancouver BC V6J 3J9",(604) 737-1551,8.2,True
245,Bard on the Beach Theatre Society,719,"1100 Chestnut St, Vancouver BC V6J 3J9",(604) 737-1551,8.2,True
219,Bard on the Beach Theatre Society,728,"1100 Chestnut St, Vancouver BC V6J 3J9",(604) 737-1551,8.2,True
263,Bard on the Beach Theatre Society,892,"1100 Chestnut St, Vancouver BC V6J 3J9",(604) 737-1551,8.2,True
184,Bard on the Beach Theatre Society,650,"1100 Chestnut St, Vancouver BC V6J 3J9",(604) 737-1551,8.2,True
181,Bard on the Beach Theatre Society,960,"1100 Chestnut St, Vancouver BC V6J 3J9",(604) 737-1551,8.2,True
268,Bard on the Beach Theatre Society,899,"1100 Chestnut St, Vancouver BC V6J 3J9",(604) 737-1551,8.2,True
159,Bard on the Beach Theatre Society,970,"1100 Chestnut St, Vancouver BC V6J 3J9",(604) 737-1551,8.2,True


In [ ]:
From this result, the data duplicates have to be dropped if longitude and latitude match,
even though the distance is different, this way we are left with unique values and we can get the top 10 values.
The result would mimick yelp values